In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np

# Paths
base_path = "/content/drive/MyDrive/Autism Images"
roi_path = "/content/drive/MyDrive/ROI_Images"
folders = ["TCImages", "TSImages"]

# Create output folders
for folder in folders:
    os.makedirs(os.path.join(roi_path, folder), exist_ok=True)

def extract_roi_only(img_path):

    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect non-black pixels
    coords = np.column_stack(np.where(gray > 5))

    if coords.size == 0:
        return None

    # Bounding box
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)

    # Add padding
    pad = 20
    y_min = max(y_min - pad, 0)
    y_max = min(y_max + pad, img.shape[0])
    x_min = max(x_min - pad, 0)
    x_max = min(x_max + pad, img.shape[1])

    roi = img[y_min:y_max, x_min:x_max]

    # Resize to 224 (for CNN)
    roi_resized = cv2.resize(roi, (224, 224))

    return roi_resized


# Process dataset
for folder in folders:
    input_folder = os.path.join(base_path, folder)
    output_folder = os.path.join(roi_path, folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):

            img_path = os.path.join(input_folder, filename)
            roi_img = extract_roi_only(img_path)

            if roi_img is not None:
                save_path = os.path.join(output_folder, filename)
                cv2.imwrite(save_path, roi_img)

print("All images cropped with ROI only and saved successfully!")


All images cropped with ROI only and saved successfully!
